# Single Subject Segmentation Q1K

In [1]:
# import packages
import q1k_sync_tools as qst
import mne
mne.viz.set_browser_backend('qt')
%matplotlib qt
import pandas as pd
pd.options.mode.chained_assignment = None
import pylossless as ll
import numpy as np
import glob

<frozen importlib._bootstrap>:219: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted


Using qt as 2D backend.


In [2]:
# set file variables
#subject_raw_eeg = glob.glob('sourcedata/020_1/020_1_go_*.mff')
subject_raw_eeg = 'sourcedata/029_1/029_1_eeg/Q1K_AudSS_Marine_20231214_021607.mff'
#subject_raw_et = 'sourcedata/029_1/029_1_eyetracking/012_ap_1.asc'
subject_derivative = 'output_ap/derivatives/pylossless/sub-029/eeg/sub-029_task-ap_eeg.edf'
task_code = 'ap'

In [3]:
subject_raw_eeg

'sourcedata/029_1/029_1_eeg/Q1K_AudSS_Marine_20231214_021607.mff'

In [3]:
# Load EEG recording
eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)

# Get event info from source recording
eeg_events = mne.find_events(eeg_raw, shortest_event=1)
eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)



# Below is for syncing EEG and ET sessions then combining the data.

# Infer new events and labels from EEG paradigm procedure
#eeg_events, eeg_stims, eeg_iti, eeg_event_dict = qst.eeg_event_test(eeg_events, eeg_event_dict, task_name=task_code)

# Read eyetracking data and extact events
#et_raw = mne.io.read_raw_eyelink(subject_raw_et)
#et_annot_events, et_annot_event_dict = mne.events_from_annotations(et_raw)
#et_raw_df = et_raw.to_data_frame()

# Infer new events from paradigm procedure
#et_event_raw_df, et_events, et_stims, et_iti = qst.et_event_test(et_raw_df, task_name=task_code)

# Meld both ET and EEG together into new raw structure
#eeg_et_raw = qst.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims)

# Using the melded raw, build final event objects
#eeg_et_events = mne.find_events(eeg_et_raw, shortest_event=1)
#eeg_et_event_dict = qst.get_event_dict(eeg_et_raw, eeg_et_events)

# Free up memory
#del et_event_raw_df
#del et_events
#del et_stims
#del et_iti
#del et_raw_df
#del et_annot_events
#del et_annot_event_dict
#del eeg_events
#del eeg_stims
#del eeg_iti
#del eeg_event_dict

Reading EGI MFF Header from /home/james/q1k/pilot/q1k-external-pilot/sourcedata/029_1/029_1_eeg/Q1K_AudSS_Marine_20231214_021607.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...


/tmp/ipykernel_9311/3173740615.py:5: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  eeg_events = mne.find_events(eeg_raw, shortest_event=1)


2709 events found
Event IDs: [1 2 3 4 5 6]


In [4]:
# Build initial state; nothing applied yet
ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(subject_derivative)

Extracting EDF parameters from /home/james/q1k/pilot/q1k-external-pilot/output_ap/derivatives/pylossless/sub-029/eeg/sub-029_task-ap_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from output_ap/derivatives/pylossless/sub-029/eeg/sub-029_task-ap_events.tsv.
Reading channel info from output_ap/derivatives/pylossless/sub-029/eeg/sub-029_task-ap_channels.tsv.
Reading electrode coords from output_ap/derivatives/pylossless/sub-029/eeg/sub-029_space-CapTrak_electrodes.tsv.
Reading output_ap/derivatives/pylossless/sub-029/eeg/sub-029_task-ap_ica1_ica.fif ...
Now restoring ICA solution ...
Ready.
Reading output_ap/derivatives/pylossless/sub-029/eeg/sub-029_task-ap_ica2_ica.fif ...


/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/pylossless/pipeline.py:1118: RuntimeWarning: The number of channels in the channels.tsv sidecar file (136) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/pylossless/pipeline.py:1118: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN4, DIN5, STI 014, TSYN, ae06, ae40, dbrk
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Now restoring ICA solution ...
Ready.


In [ ]:
# Plot channel data only
mne.viz.plot_raw(ll_state.raw, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# Plot comonents
ll_state.ica2.plot_sources(ll_state.raw)

In [ ]:
# Plot topos
ll_state.ica2.plot_components(inst=ll_state.raw)

In [5]:
# Merge marks down to bads (aka manual)
ll_qcr = ll_state.raw.copy()
manual = []
for flag_type in ll_state.flags['ch']:
    manual.extend(ll_state.flags['ch'][flag_type])
print(ll_state.flags['ch'])
ll_qcr.info['bads'].extend(manual)
ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))

# Read the ICLabel info from file and add to exclude
df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)

# Load the data and apply the ICA
ll_qcr.load_data()
ll_state.ica2.apply(ll_qcr)
ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=100.0)
ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
ll_qcr

{'ch_sd': array(['E8'], dtype=object), 'rank': array(['E73'], dtype=object)}
Reading 0 ... 677999  =      0.000 ...   677.999 secs...


Applying ICA to Raw instance
    Transforming to ICA space (122 components)
    Zeroing out 42 ICA components
    Projecting back using 123 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 s)



/tmp/ipykernel_9311/865797808.py:17: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=100.0)
/tmp/ipykernel_9311/865797808.py:17: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=100.0)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    6.3s finished
/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()
/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()


<RawEDF | sub-029_task-ap_eeg.edf, 129 x 678000 (678.0 s), ~667.5 MB, data loaded>

In [ ]:
# Plot cleaned data
mne.viz.plot_raw(ll_qcr, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# Take a look at the event dictionairy
eeg_event_dict

In [6]:
# Relabel condition vars for niceness
eeg_event_dict['fq06hz/target/disp/ae06'] = eeg_event_dict.pop('ae06')
eeg_event_dict['fq40hz/target/disp/ae40'] = eeg_event_dict.pop('ae40')

In [7]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1, tmax=2.0, on_missing='warn', event_repeated='drop')

Not setting metadata
2709 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated


In [8]:
# Check trial counts
epochs[['fq06hz', 'fq40hz']]

/home/james/q1k/pilot/q1k-external-pilot/eeg-env/lib/python3.8/site-packages/IPython/core/formatters.py:344: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  return method()


Number of events,160
Events,fq06hz/target/disp/ae06: 80fq40hz/target/disp/ae40: 80
Time range,-1.000 – 2.000 s
Baseline,-1.000 – 0.000 s


In [19]:
epochs.save(f'erp_fif_files/ap/004_1_ap_ep.fif', overwrite=True)

Using data from preloaded Raw for 249 events and 3001 original time points ...


/tmp/ipykernel_3743/504302322.py:1: RuntimeWarning: This filename (erp_fif_files/ap/004_1_ap_ep.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(f'erp_fif_files/ap/004_1_ap_ep.fif', overwrite=True)


71 bad epochs dropped
Using data from preloaded Raw for 1 events and 3001 original time points ...
Using data from preloaded Raw for 178 events and 3001 original time points ...


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 2394, resource id: 0, major code: 140 (Unknown), minor code: 20


In [9]:
# Plot ERPs
evokeds = {'fq06hz': epochs['fq06hz'].average(), 'fq40hz': epochs['fq40hz'].average()}
mne.viz.plot_compare_evokeds(evokeds, picks=['E6'], combine='mean')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_9311/3105607422.py:3: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(evokeds, picks=['E6'], combine='mean')


combining channels using "mean"
combining channels using "mean"


[<Figure size 800x600 with 2 Axes>]

In [15]:
mne.write_evokeds(f'erp_fif_files/ap/024_1_ap_ave.fif',list(evokeds.values()), overwrite=True)

In [10]:

mask = np.isin(eeg_events[:,2],[eeg_event_dict['fq40hz/target/disp/ae40']])
eeg_events = eeg_events[mask]


epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-2.0, tmax=3.0, on_missing='warn', event_repeated='merge')

freqs = np.array([5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90])
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(
    epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

Not setting metadata
80 matching events found
Setting baseline interval to [-2.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 80 events and 5001 original time points ...


/tmp/ipykernel_9311/1521413859.py:5: RuntimeWarning: No matching events found for TSYN (event id 1)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-2.0, tmax=3.0, on_missing='warn', event_repeated='merge')
/tmp/ipykernel_9311/1521413859.py:5: RuntimeWarning: No matching events found for dbrk (event id 4)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-2.0, tmax=3.0, on_missing='warn', event_repeated='merge')
/tmp/ipykernel_9311/1521413859.py:5: RuntimeWarning: No matching events found for DIN4 (event id 5)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-2.0, tmax=3.0, on_missing='warn', event_repeated='merge')
/tmp/ipykernel_9311/1521413859.py:5: RuntimeWarning: No matching events found for DIN5 (event id 6)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-2.0, tmax=3.0, on_missing='warn', event_repeated='merge')
/tmp/ipykernel_9311/1521413859.py:5: RuntimeWarning: No matching events foun

6 bad epochs dropped


/tmp/ipykernel_9311/1521413859.py:9: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  power, itc = mne.time_frequency.tfr_morlet(
/tmp/ipykernel_9311/1521413859.py:9: RuntimeWarning: The MNE-Python config file (/home/james/.mne/mne-python.json) is not a valid JSON file and might be corrupted
  power, itc = mne.time_frequency.tfr_morlet(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 123 out of 123 | elapsed:   57.9s finished


In [11]:
power.plot(picks='E6', baseline=(-0.5, 0), mode="logratio", title='power')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying baseline correction (mode: logratio)


[<Figure size 640x480 with 2 Axes>]

In [13]:
itc.plot(picks='E6')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
No baseline correction applied


[<Figure size 640x480 with 2 Axes>]